In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime
years = [2016 , 2017 , 2018]
URLPIN = "https://us.soccerway.com"

In [2]:
class Match:
    def __init__(self , teamA , teamB , info , date ,goals,competition):
        self.teamA = teamA
        self.teamB = teamB
        self.info = info
        self.date = date
        self.goals = goals
        self.competition = competition
        self.playersTeamA = list()
        self.playersTeamB = list()
    def __eq__(self,other):
        return self.info == other.info
    def __repr__(self):
        return self.teamA+"-"+self.teamB+" Fecha: "+self.date.strftime("%d/%m/%Y")
    def __hash__(self):
        return 0
    

In [3]:

class Countries:
    def __init__(self, name,info):
        self.name = name
        self._info = info
        self.matches = set()
    def __eq__(self,other):
        return self.name == other.name
    def __repr__(self):
        return self.name
    def __str__(self):
        return self.name
    def __hash__(self):
        return 0
    
class Player:
    def __init__(self, name , info):
        self.name = name
        self.info = info
    def __repr__(self):
        return self.name
    def __str__(self):
        return self.name            
    

In [4]:
def getGoal(text):
    text = text.text.strip()
    splitText = None 
    if "P" in text:
        splitText = [int(goal) for goal in text.split() if goal not in ["P" ,"-"]]
    else :
        splitText = [int(goal) for goal in text.split("-")]
    return splitText[0] ,splitText[1]
def returnWebObject(url):
    return BeautifulSoup(urlopen(url) , "lxml")

def getDateTime(row):
    date = datetime.fromtimestamp(int(row["data-timestamp"]))
    if date is None:
        return None
    if date.year not in years :
        return None
    if date > datetime.now():
        return None
    return date

def getInfoMatches(web):
    table = web.find("table" , {"class":"matches "}).find("tbody")
    countrie = set()
    for classes in ["even match no-date-repetition" ,"odd match no-date-repetition"]:
        for row in table.findAll("tr",{"class":classes}):
            date = getDateTime(row)
            if date is None:
                continue
            linkInfoMatch = row.find("td",{"class":"score-time score"})
            try:
                goals = getGoal(linkInfoMatch)
            except:
                print("error Goles")
                continue
            tagTeamA = row.find("td", {"class":"team team-a "}).find("a").text.strip()
            tagTeamB = row.find("td",{"class":"team team-b "}).find("a").text.strip()
            competition = row.find("td",{"class":"competition"}).find("a")["title"]
            info = linkInfoMatch.find("a")["href"]
            countrie.add(Match(tagTeamA, tagTeamB, info, date , goals,competition))
    return countrie

def getPlayers(info):
    playersTeamA = list()
    playersTeamB = list()
    infoPlayers = returnWebObject(URLPIN + info)
    containers = infoPlayers.find("div",{"class":"combined-lineups-container"}) 
    i = 0
    try:
        for tables in ["container left" ,"container right"]:
            table = containers.find("div",{"class":tables}).find("table",{"class":"playerstats lineups table"})
            for row in table.findAll("tr")[1:12]:
                playerInfo = row.find("td",{"class":"player large-link"}).find("a",href=True)
                if i == 0:
                    playersTeamA.append(Player(playerInfo.text.strip(), playerInfo["href"]))
                else :
                    playersTeamB.append(Player(playerInfo.text.strip(), playerInfo["href"]))
            i = i + 1
    except:
        print(URLPIN + info)
        return list() , list()
    return playersTeamA, playersTeamB

In [5]:
countries = set()
contains = returnWebObject("https://us.soccerway.com/international/world/world-cup/2018-russia/group-stage/r32579/?ICID=TN_02_03_01")
for group in contains.findAll("div",{"class":"block clearfix block_competition_tables_group-wrapper"}):
    table = group.find("table").find("tbody")
    for row in table.findAll("tr"):
        info = row.find("td",{"class":"text team large-link"}).find("a")
        name = info.text.strip()
        countries.add(Countries(name,info["href"]))
countries

{Argentina,
 Australia,
 Belgium,
 Brazil,
 Colombia,
 Costa Rica,
 Croatia,
 Denmark,
 Egypt,
 England,
 France,
 Germany,
 Iceland,
 Iran,
 Japan,
 Korea Republic,
 Mexico,
 Morocco,
 Nigeria,
 Panama,
 Peru,
 Poland,
 Portugal,
 Russia,
 Saudi Arabia,
 Senegal,
 Serbia,
 Spain,
 Sweden,
 Switzerland,
 Tunisia,
 Uruguay}

In [6]:
linkSet = set()
for countrie in countries:
    linkMatches = URLPIN+countrie._info+"matches/"
    web = returnWebObject(linkMatches)
    countrie.matches = getInfoMatches(web)
    for match in countrie.matches:
        if match.info in linkSet:
            continue
        else:
            linkSet.add(match.info)
            match.playersTeamA , match.playersTeamB = getPlayers(match.info)

error Goles
error Goles
error Goles
error Goles
error Goles
error Goles
https://us.soccerway.com/matches/2017/03/18/world/friendlies/iran/iraq/2423044/
https://us.soccerway.com/matches/2016/11/10/world/friendlies/iran/papua-new-guinea/2367846/
https://us.soccerway.com/matches/2016/08/19/world/hybrid-friendlies/spal-1907/iran/2339067/
error Goles
error Goles
error Goles
https://us.soccerway.com/matches/2016/03/25/africa/africa-cup-of-nations-qualification/nigeria/egypt/2038209/
https://us.soccerway.com/matches/2017/08/13/africa/african-nations-championship/egypt/morocco/2410355/
https://us.soccerway.com/matches/2017/06/12/africa/africa-cup-of-nations-qualification/tunisia/egypt/2412135/
https://us.soccerway.com/matches/2017/08/18/africa/african-nations-championship/morocco/egypt/2410356/
https://us.soccerway.com/matches/2016/02/27/world/friendlies/egypt/burkina-faso/2207539/
https://us.soccerway.com/matches/2016/03/29/africa/africa-cup-of-nations-qualification/egypt/nigeria/2038211/
htt

In [7]:
import pandas as pd
#infoDataFrame = pd.DataFrame()
listRows = list()
for countrie in countries:
    for match in countrie.matches:
        row = dict()
        row["EquipoA"] = match.teamA
        row["EquipoB"] = match.teamB
        row["GolesA"] =  match.goals[0]
        row["GolesB"] = match.goals[1]
        row["Comp"] = match.competition
        row["JugadoresA"] = match.playersTeamA
        row["JugadoresB"] = match.playersTeamB
        row["Info"] = match.info
        listRows.append(row)
infoDataFrame = pd.DataFrame(listRows)
        

In [8]:
infoDataFrame.drop_duplicates(subset=["Info"],keep='first',inplace=True)

In [9]:
infoDataFrame.to_csv("PartidosGoles.csv",index=False)

In [ ]:
countrieDF = pd.DataFrame(columns = ["Pais", "Link"])
for countrie in countries:
    rowCountri = dict()
    rowCountri["Pais"] = countrie.name
    rowCountri["Link"] = URLPIN+countrie._info
    countrieDF.loc[len(countrieDF)] = rowCountri
countrieDF

In [ ]:
import pandas as pd
linkSet = set()

playerDF = pd.DataFrame(columns = ["Nombre","Link","Pais"])
for countrie in countries:
    for match in countrie.matches:
        if match.info  in linkSet:
            continue
        else:
            linkSet.add(match.info)
            for player in match.playersTeamA:
                if match.teamA not in list(countrieDF["Pais"]):
                    break
                else:
                    rowPlayer = dict()
                    rowPlayer["Pais"] = match.teamA
                    rowPlayer["Nombre"] = player.name
                    rowPlayer["Link"] = URLPIN+player.info
                    playerDF.loc[len(playerDF)] = rowPlayer
            for player in match.playersTeamB:
                
                if match.teamB not in list(countrieDF["Pais"]):
                    break
                else:
                    rowPlayer = dict()
                    rowPlayer["Pais"] = match.teamB
                    rowPlayer["Nombre"] = player.name
                    rowPlayer["Link"] = URLPIN+player.info
                    playerDF.loc[len(playerDF)] = rowPlayer
           